 <font color=blue size=4> 
Case 2 - Integração com API Spotify
 </font>

In [ ]:
import os
import json
import base64
import requests
from google.cloud import secretmanager
from google.cloud import bigquery

# Função para configurar o Secret Manager e obter as credenciais do Spotify
def get_secret(secret_id):
    client = secretmanager.SecretManagerServiceClient()
    secret_name = f"projects/{os.environ['GOOGLE_CLOUD_PROJECT']}/secrets/{secret_id}/versions/latest"
    response = client.access_secret_version(request={"name": secret_name})
    return json.loads(response.payload.data.decode("UTF-8"))

# Obter as credenciais do Spotify armazenadas no Secret Manager
spotify_credentials = get_secret("spotify-credentials")
client_id = spotify_credentials['client_id']
client_secret = spotify_credentials['client_secret']

# Função para obter o token de acesso do Spotify
def acess_token_spotify(client_id, client_secret):
    auth_url = "https://accounts.spotify.com/api/token"
    credentials = f"{client_id}:{client_secret}"
    encoded_credentials = base64.b64encode(credentials.encode()).decode()

    headers = {
        "Authorization": f"Basic {encoded_credentials}",
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "client_credentials"
    }

    response = requests.post(auth_url, headers=headers, data=data)
    if response.status_code != 200:
        print(f"Erro ao obter token: {response.text}")
        return None
    return response.json().get("access_token")

# Função para buscar podcasts
def buscar_todos_os_podcasts(query, access_token):
    print("Buscando podcasts...")
    podcasts = []
    offset = 0
    total = 1000  # Defina o total de podcasts que você espera obter

    while offset < total:
        url = f"https://api.spotify.com/v1/search?query={query}&type=show&offset={offset}&limit=20"
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        response = requests.get(url, headers=headers)

        if response.status_code != 200:
            print(f"Erro na busca de podcasts: {response.text}")
            break

        data = response.json()
        if data.get("shows"):
            podcasts.extend(data["shows"]["items"])
            total = data["shows"]["total"]  # Atualiza o total a partir da resposta
            offset += 20
        else:
            break

    return podcasts

# Função principal
def main():
    # Configurações
    os.environ['GOOGLE_CLOUD_PROJECT'] = 'data-engineering-3254'
    secret_id = "spotify-credentials"
    dataset_id = "spotify_podcasts"
    podcasts_table_id = "podcasts"

    # Obter o token de acesso do Spotify
    access_token = acess_token_spotify(client_id, client_secret)
    if access_token is None:
        print("Falha ao obter token de acesso.")
        return

    # Buscar todos os podcasts
    podcasts = buscar_todos_os_podcasts("data hackers", access_token)
    print(f"Total de podcasts encontrados: {len(podcasts)}")

    # Adicionar lógica para inserir dados no BigQuery
    if podcasts:
        client = bigquery.Client()
        table_ref = client.dataset(dataset_id).table(podcasts_table_id)

        # Criação de uma lista de linhas a serem inseridas
        rows_to_insert = []
        for podcast in podcasts:
            row = {
                "id": podcast['id'],
                "name": podcast['name'],
                "description": podcast.get('description', ''),
                "publisher": podcast['publisher'],
                # Adicione outros campos que você deseja armazenar
            }
            rows_to_insert.append(row)

        # Inserir dados na tabela
        errors = client.insert_rows_json(table_ref, rows_to_insert)
        if errors == []:
            print("Dados inseridos com sucesso no BigQuery.")
        else:
            print(f"Erros ao inserir dados: {errors}")
    else:
        print("Nenhum podcast encontrado.")

# Chamada da função principal
if __name__ == "__main__":
    main()